In [1]:
from skyfield.api import load, Topos

# Load the TLE data from the local file
satellites = load.tle_file('stations.txt')  # Replace with the correct path
satellite = satellites[0]  # Select the first satellite (ISS)

# Define the ground station's location (Copenhagen, Denmark)
ground_station = Topos(latitude_degrees=55.6761, longitude_degrees=12.5683)

# Create a timescale and ask for the current time
ts = load.timescale()
t = ts.now()

# Compute the satellite's position relative to the ground station
difference = satellite - ground_station

# Get the satellite's altitude and azimuth relative to the observer (ground station)
topocentric = difference.at(t)
alt, az, d = topocentric.altaz()

# Check if the satellite is above the horizon (altitude > 0)
if alt.degrees > 0:
    print(f"Satellite is visible! Altitude: {alt.degrees:.2f}° Azimuth: {az.degrees:.2f}°, Distance: {d.km:.2f} km")
else:
    print(f"Satellite is not visible. Altitude: {alt.degrees:.2f}°, Azimuth: {az.degrees:.2f}°, Distance: {d.km:.2f} km")


Satellite is not visible. Altitude: -48.79°, Azimuth: 298.18°, Distance: 10137.33 km


In [10]:
from time import sleep

for i in range(1000):
    t = ts.now()
    topocentric = difference.at(t)
    alt, az, d = topocentric.altaz()
    if alt.degrees > 0:
        print(f"Satellite is visible! Altitude: {alt.degrees:.2f}° Azimuth: {az.degrees:.2f}°, Distance: {d.km:.2f} km       ", end='\r')
    else:
        print(f"Satellite is not visible. Altitude: {alt.degrees:.2f}°, Azimuth: {az.degrees:.2f}°, Distance: {d.km:.2f} km       ", end='\r')
    sleep(1)

KeyboardInterrupt: 

In [6]:
from skyfield.api import load, Topos
from datetime import datetime, timedelta

# Load the TLE data from the local file (stations.txt)
satellites = load.tle_file('stations.txt')  # Ensure the path to your TLE file is correct

# Define Copenhagen, Denmark (latitude and longitude)
ground_station = Topos(latitude_degrees=55.6761, longitude_degrees=12.5683)

# Load the Skyfield timescale
ts = load.timescale()

# Calculate when a satellite is above the horizon for a continuous time span (e.g., 3 minutes)
def get_pass_times(satellite, ground_station, start_time, end_time, min_duration=timedelta(minutes=3)):
    passes = []
    time = start_time
    last_time = None
    in_pass = False
    pass_start_time = None
    
    while time < end_time:
        # Convert the datetime to Skyfield time
        skyfield_time = ts.utc(time.year, time.month, time.day, time.hour, time.minute, time.second)
        difference = satellite - ground_station
        topocentric = difference.at(skyfield_time)
        alt, az, d = topocentric.altaz()
        
        if alt.degrees > 0:  # Satellite is above the horizon
            if not in_pass:  # We are entering a new pass
                pass_start_time = time
                in_pass = True
        else:  # Satellite is below the horizon
            if in_pass:  # We were in a pass, and now it ends
                pass_duration = time - pass_start_time
                if pass_duration >= min_duration:  # Only consider passes above the horizon longer than the minimum duration
                    passes.append((pass_start_time, time))
                in_pass = False
        
        time += timedelta(seconds=10)  # Check every 10 seconds
        
    # If the pass was still ongoing at the end of the loop, close it
    if in_pass:
        pass_duration = time - pass_start_time
        if pass_duration >= min_duration:
            passes.append((pass_start_time, time))
    
    return passes

# Start and end time for the pass predictions (next 24 hours)
start_time = datetime.utcnow()
end_time = start_time + timedelta(days=1)

# Print passes for each satellite in the TLE file
for satellite in satellites:
    passes = get_pass_times(satellite, ground_station, start_time, end_time)
    print(f"Pass times for {satellite.name}:")
    for pass_start, pass_end in passes:
        print(f"  Communication possible from {pass_start.strftime('%Y-%m-%d %H:%M:%S UTC')} to {pass_end.strftime('%Y-%m-%d %H:%M:%S UTC')}")


Pass times for ISS (ZARYA):
  Communication possible from 2024-12-18 07:00:57 UTC to 2024-12-18 07:06:47 UTC
  Communication possible from 2024-12-18 08:32:27 UTC to 2024-12-18 08:42:57 UTC
  Communication possible from 2024-12-18 10:09:07 UTC to 2024-12-18 10:18:27 UTC
  Communication possible from 2024-12-18 11:47:17 UTC to 2024-12-18 11:51:07 UTC
  Communication possible from 2024-12-19 02:58:27 UTC to 2024-12-19 03:03:17 UTC
  Communication possible from 2024-12-19 04:31:37 UTC to 2024-12-19 04:41:07 UTC
  Communication possible from 2024-12-19 06:07:07 UTC to 2024-12-19 06:17:47 UTC
Pass times for CSS (TIANHE):
  Communication possible from 2024-12-18 23:42:57 UTC to 2024-12-18 23:48:07 UTC
  Communication possible from 2024-12-19 01:17:37 UTC to 2024-12-19 01:24:47 UTC
  Communication possible from 2024-12-19 02:53:37 UTC to 2024-12-19 03:00:17 UTC
Pass times for ISS (NAUKA):
  Communication possible from 2024-12-18 07:00:57 UTC to 2024-12-18 07:06:47 UTC
  Communication possible